In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


# SELF JOINs

One of the most common use cases for self JOINs is in cases where two events occurred, one after another. As you may have noticed in the previous video, using inequalities in conjunction with self JOINs is common.

Task 1:

Modify the query from the previous video, which is pre-populated in the SQL Explorer below, to perform the same interval analysis except for the web_events table. Also:

change the interval to 1 day to find those web events that occurred after, but not more than 1 day after, another web event
add a column for the channel variable in both instances of the table in your query
You can find more on the types of INTERVALS (and other date related functionality) in the Postgres documentation here:
https://www.postgresql.org/docs/8.2/functions-datetime.html

SELECT o1.id AS o1_id,
       o1.account_id AS o1_account_id,
       o1.occurred_at AS o1_occurred_at,
       o2.id AS o2_id,
       o2.account_id AS o2_account_id,
       o2.occurred_at AS o2_occurred_at
  FROM orders o1
 LEFT JOIN orders o2
   ON o1.account_id = o2.account_id
  AND o2.occurred_at > o1.occurred_at
  AND o2.occurred_at <= o1.occurred_at + INTERVAL '28 days'
ORDER BY o1.account_id, o1.occurred_at

In [6]:
# sqlite 3 version

pd.read_sql_query(sql = '''
SELECT we1.id AS we_id,
       we1.account_id AS we1_account_id,
       we1.occurred_at AS we1_occurred_at,
       we1.channel AS we1_channel,
       we2.id AS we2_id,
       we2.account_id AS we2_account_id,
       we2.occurred_at AS we2_occurred_at,
       we2.channel AS we2_channel
  FROM web_events we1 
 LEFT JOIN web_events we2
   ON we1.account_id = we2.account_id
  AND we1.occurred_at > we2.occurred_at
  AND we1.occurred_at <= date (we2.occurred_at, '+1 day')
ORDER BY we1.account_id, we2.occurred_at
;
''', con=conn)[:50]

,we_id,we1_account_id,we1_occurred_at,we1_channel,we2_id,we2_account_id,we2_occurred_at,we2_channel
0,2,1001,2015-11-05 03:08:26,direct,NaN,NaN,None,None
1,3,1001,2015-12-04 03:57:24,direct,NaN,NaN,None,None
2,4,1001,2016-01-02 00:55:03,direct,NaN,NaN,None,None
3,5,1001,2016-02-01 19:02:33,direct,NaN,NaN,None,None
4,6,1001,2016-03-02 15:15:22,direct,NaN,NaN,None,None
5,7,1001,2016-04-01 10:58:55,direct,NaN,NaN,None,None
6,9,1001,2016-05-31 20:53:47,direct,NaN,NaN,None,None
7,10,1001,2016-06-30 12:09:45,direct,NaN,NaN,None,None
8,11,1001,2016-07-30 03:06:26,direct,NaN,NaN,None,None
9,12,1001,2016-08-28 06:42:42,direct,NaN,NaN,None,None


Solution:

SELECT we1.id AS we_id,
       we1.account_id AS we1_account_id,
       we1.occurred_at AS we1_occurred_at,
       we1.channel AS we1_channel,
       we2.id AS we2_id,
       we2.account_id AS we2_account_id,
       we2.occurred_at AS we2_occurred_at,
       we2.channel AS we2_channel
  FROM web_events we1 
 LEFT JOIN web_events we2
   ON we1.account_id = we2.account_id
  AND we1.occurred_at > we2.occurred_at
  AND we1.occurred_at <= we2.occurred_at + INTERVAL '1 day'
ORDER BY we1.account_id, we2.occurred_at